In [ ]:
!pip install python-telegram-bot==13.7 requests yt-dlp

In [ ]:
from telegram.ext import Updater, CommandHandler , MessageHandler, Filters
import yt_dlp
import os

Token = "" # Telegram bot Token


def start(update , context):
  update.message.reply_text("Send your video's YouTube link.")

def send_video(update, video_path):
    if os.path.exists(video_path):
        with open(video_path, 'rb') as video_file:
            update.message.reply_video(video_file)  # Send the video file
    else:
        update.message.reply_text("Error: Video file not found!")
    print("Sent!")

def download_video(update , context):
    url = update.message.text.strip()
    if "youtu.be" not in url:
        update.message.reply_text("Please send a valid YouTube link.")
        return

    if not os.path.exists('downloads'):
        os.makedirs('downloads')

    try:
        ydl_opts = {
            'format': 'best',
            'outtmpl': 'downloads/%(id)s.%(ext)s',
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

        info_dict = ydl.extract_info(url, download=False)
        video_path = ydl.prepare_filename(info_dict)

        send_video(update, video_path)

        return f"Download complete! File saved as '{video_path}'."
    except Exception as e:
        return f"An error occurred: {e}"



updater = Updater(Token, use_context=True)
dispatch = updater.dispatcher

dispatch.add_handler(CommandHandler("start" , start))

dispatch.add_handler(MessageHandler(Filters.text , download_video))


updater.start_polling()
updater.idle()